In [331]:
import numpy as np
import inspect
import regex
import json
from typing import List, Callable, Union
import math

In [333]:
ignored_characters = regex.compile("[^A-Za-z0-9\,\;]")

def compute_prediction_score(truth: List[int], solution: Callable[[int], int]) -> float:
    prediction_score = 0
    for i in range(len(truth)):
        #print("truth of i",truth[i], "soluntion:")
        distance = np.abs(truth[i] - solution(i))
        #print(distance)
        prediction_score += (10-distance) if distance < 10 else 0
    prediction_score /= len(truth)
    return prediction_score

def evaluate(truth: List[int], solution: Callable[[int], int]) -> float:
    """ 
    Returns the loss of a solution 
    :param truth: array of ground truth hashes
    :param solution:  solution function, which takes an index and returns the 
                      predicted hash

    :return: the score as a float in the range [0,10]
    """
    prediction_score = compute_prediction_score(truth, solution)
    
    print("Average prediction score: ", prediction_score)
    
    # remove `def function_name(x):` from the source
    source = inspect.getsource(solution)
    source = source[source.index(":")+1:]

    length_score = len(source) - len(ignored_characters.findall(source))
    print(length_score)
    length_score -= 100
    print(length_score)
    length_score /= 100
    print(length_score)
    length_score = length_score if length_score > 1.0 else 1.0
    
    print("Length score: ", length_score)
    
    score = prediction_score / length_score
    print("Final score: ", score)
    return score

In [335]:
!curl -O https://x80-public.s3.eu-west-3.amazonaws.com/hashes.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4496  100  4496    0     0  14936      0 --:--:-- --:--:-- --:--:-- 15036


In [334]:
import json
with open("hashes.json", "r") as f:
  hashes = json.loads(f.read())
hashes_easy, hashes_medium, hashes_hard, over_9000 = hashes["hashes_easy"], hashes["hashes_medium"], hashes["hashes_hard"], hashes["over_9000"]

# Solution Easy

# Approach
### After viewing the hash_easy series it was clear that after every 5 index values the series was repeating with some pattern.
### Index 0 had value of 1, followed  by  fixed difference  of 3 for index 1, 2 and 3 and  -7 for index 4 , -2 for index 5 ,  repeating this pattern over full lenght of the given array. 
### The below code recursively calculates the required output. 

In [352]:
def solution_easy (x):
    
    num= 0
    if(x > 0):
        if((x+1)%5==0):
            num = -7
        elif (x%5==0):
            #print(x,"x value")
            num = -2         
        else:
            #print("in else for x =",x)
            num=3
        result= (solution_easy(x-1)+num)
        return result 
    
    else:
        return 1
# you only get the `hashes_test`
hashes_test = [solution_easy(x) for x in range(128)]
print(hashes_test)
#solution(5)

[1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7, 10, 3, 1, 4, 7]


In [353]:
evaluate(hashes_easy, solution_easy)

Average prediction score:  10.0
113
13
0.13
Length score:  1.0
Final score:  10.0


10.0

# Solution Medium

# Approach
## In this series it was found that 7 value was added to repeatedly for maximum of time with 10 value added after particular interval. Only the the addition of 7 was in specific pattern like 7 was added progresively for first 6 index position.
### For 7th index 10 was added to value of index [2] value and later again 7 was added progresively for 5 times.
### Addition of 7 five time and again adding 10 or the 6th time process is done for 3 repeatation and once 7 is added for 6 times followed by same 5 times 7 addition process.
#### To handle this special case of adding 7 value 6 times after interval of 3 times ( 10 value addition) code was written with few declared variable like 
#### step : which holds the value of how many time 10 was added
#### num: which hold the value of how many time 7 was added.
#### flag: which hold value for logic to flip the value of step between  3 or 1 
#### inputA : array hold the basic value of the list since they are straight forward multiple of 7 from 0 to 6, actual series pattern started from index 7 

In [355]:
def solution_medium1(x):
    inputA =[0,7,14,21,28,35,42,24]
    step = 3
    num =5
    flag =1
   
    for z in range (8,x+1):
        
        if num>0:
            inputA.append(inputA[z-1]+7)
            num-=1
        else:
            inputA.append(inputA[z-5]+10)
            step-=1
            
            if (step==0):
                if(flag==1):
                    num=6
                    step=1
                    flag= 0
                else:
                    num=5
                    step=3
                    flag=1
            else:
                num=5
   
    return inputA[x]

In [356]:
evaluate(hashes_medium, solution_medium1)

Average prediction score:  10.0
185
85
0.85
Length score:  1.0
Final score:  10.0


10.0

# Solution Hard

# Approach
## In this series it was observed that every 5th index had a relation with its preceding 4th index value. Also this series had a break in its pattern right in the midpoint of total given length of array at 128 index and from 129 again the same pattern was applied. In order to achieve correct output I have hard coded calculation for 128th index . Further to this if this type of pattern breaker is applied for every 128th value, and knowing the full length of the array we can remove the hard code and define a logic to calculate value accordingly. 

### Here I have applied one calculation logic to get the value which is added repeatedly to every 5th index in realtion to its preceding 4th index value. It was found that value is in multiple of ( 40 and/or 24) was added to generate the every 5th index value and for every succedding value next odd number was added.

### For all other value a pattern of +2 to the index value was observed.
### Used mostly mod operators to develop the logic to get desired output.

In [350]:
def solution_hard(r):
    if(r==0):
        return 127
    if (r%4==0):
        dD = math.floor(r/4)
        sD= math.floor(dD/2)
        n40= dD-sD
        num= n40*40 + sD*24
        if (r==128):
            num= num -1
            return (solution_hard(r-1)+num)

        return (solution_hard(r-4)+num)
    
    elif ((r-1)%4==0):
        od=0
        for abc in range(r+1, r+3):
            if(abc%2!=0):
                od=abc
                break
        return (solution_hard(r-1)+od)
    else :
        return (solution_hard(r-2)+2)
            
solution_hard(128)     

17279

In [351]:
evaluate(hashes_hard, solution_hard)

Average prediction score:  10.0
216
116
1.16
Length score:  1.16
Final score:  8.620689655172415


8.620689655172415